In [218]:
import torch
import torch.nn as nn 
import numpy as  np 
import torch.nn.functional as F 

In [219]:
# Define RNN model 

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dense_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, dense_size)

    # Corrected method name
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Take the last timestep output for predictions
        return out


input_size = 1
hidden_size = 2
dense_size = 1

model = SimpleRNN(input_size,hidden_size,dense_size)
torch.manual_seed(0)  # Set seed for reproducibility

In [220]:
model

SimpleRNN(
  (rnn): RNN(1, 2, batch_first=True)
  (fc): Linear(in_features=2, out_features=1, bias=True)
)

In [221]:
# Extracting weights
wx = model.rnn.weight_ih_l0.data.numpy()  # Input-hidden weights
wh = model.rnn.weight_hh_l0.data.numpy()  # Hidden-hidden weights
bh = model.rnn.bias_hh_l0.data.numpy()    # Hidden bias
wy = model.fc.weight.data.numpy()       # Fully connected layer weights
by = model.fc.bias.data.numpy()           # Fully connected layer bias

print('wx =', wx)
print('wh =', wh)
print('bh =', bh)
print('wy =', wy)
print('by =', by)

wx = [[-0.00529398]
 [ 0.3793229 ]]
wh = [[-0.58198076 -0.5203875 ]
 [-0.27234524  0.18961589]]
bh = [-0.06275152  0.18710934]
wy = [[-0.21369691 -0.13899271]]
by = [-0.6755334]


In [222]:
x = np.array([1,2,3])
print("BEFORE RESHAPING",x)
x_tensor = torch.tensor(x,dtype=torch.float32).reshape(1,3,1)
print("AFTER RESHAPING",x_tensor)
print(" 1st dimension MEANS BATCH SIZE \n 2nd dimension means Sequence lenght \n 3rd dimension means input size" )

BEFORE RESHAPING [1 2 3]
AFTER RESHAPING tensor([[[1.],
         [2.],
         [3.]]])
 1st dimension MEANS BATCH SIZE 
 2nd dimension means Sequence lenght 
 3rd dimension means input size


In [223]:
y_pred = model(x_tensor).detach().numpy()
print("PREDICTION FROM  MODEL" ,y_pred)

PREDICTION FROM  MODEL [[-0.74345857]]


In [224]:
m = hidden_size
h0 = np.zeros(m)  # Initial hidden state
h1 = np.dot(x[0], wx) + h0 + bh
h2 = np.dot(x[1], wx) + np.dot(h1,wh) + bh
h3 = np.dot(x[2], wx) + np.dot(h2,wh) + bh
print(h3.shape, wy.shape)
o3 = np.dot(h3, wy.T) + by
print('h1 =', h1,'h2 =', h2,'h3 =', h3)
print("Prediction from network ", y_pred)
print("Prediction from our computation ", o3)

(2, 2) (1, 2)
h1 = [[-0.0680455   0.18181536]
 [ 0.31657137  0.56643222]] h2 = [[-0.08325486  0.24640648]
 [ 0.35739069  0.88841989]] h3 = [[-0.09728837  0.26127476]
 [ 0.62526571  1.30755489]]
Prediction from network  [[-0.74345857]]
Prediction from our computation  [[-0.69105848]
 [-0.99089136]]
